In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/air-quality/global_air_quality_data_10000_date_year_2022.csv
/kaggle/input/country-data/Countries.csv


In [18]:
import pandas as pd
import numpy as np

AIR_PATH = "/kaggle/input/air-quality/global_air_quality_data_10000_date_year_2022.csv"
CTY_PATH = "/kaggle/input/country-data/Countries.csv"
OUT_DIR = "/kaggle/working/"


In [4]:
air = pd.read_csv(AIR_PATH)
cty = pd.read_csv(CTY_PATH)

print("AIR:", air.shape)
print("COUNTRIES:", cty.shape)

print("\nAIR nulls:", air.isna().sum().sum())
print("AIR duplicates:", air.duplicated().sum())

print("\nCOUNTRIES key duplicates (Country Name, Year):",
      cty.duplicated(subset=["Country Name", "Year"]).sum())

cty.isna().sum().sort_values(ascending=False).head(10)


AIR: (10000, 12)
COUNTRIES: (5106, 25)

AIR nulls: 0
AIR duplicates: 0

COUNTRIES key duplicates (Country Name, Year): 161


R&D                           1518
Unemployment                   667
Ease of Doing Business         598
Health Expenditure (% GDP)     575
Health Expenditure             575
Inflation Rate                 552
Export (% GDP)                 460
Net Trade                      460
Import                         460
Export                         460
dtype: int64

In [6]:
air.columns = [c.strip() for c in air.columns]
air["City"] = air["City"].astype(str).str.strip()
air["Country"] = air["Country"].astype(str).str.strip()

air["Date"] = pd.to_datetime(air["Date"], errors="coerce")
assert air["Date"].isna().sum() == 0, "Date parsing failed for some rows"

air["Year"] = air["Date"].dt.year.astype("Int64")
air["Month"] = air["Date"].dt.month.astype("Int64")
air["MonthName"] = air["Date"].dt.month_name()
air["Day"] = air["Date"].dt.day.astype("Int64")

def flag_out_of_range(series, lo=None, hi=None):
    s = pd.to_numeric(series, errors="coerce")
    flag = pd.Series(False, index=series.index)
    if lo is not None: flag |= s < lo
    if hi is not None: flag |= s > hi
    return flag

air["Flag_Humidity_Range"] = flag_out_of_range(air["Humidity"], 0, 100)
air["Flag_Wind_Negative"] = flag_out_of_range(air["Wind Speed"], 0, None)

print("Humidity out of range:", int(air["Flag_Humidity_Range"].sum()))
print("Negative wind speed:", int(air["Flag_Wind_Negative"].sum()))



Humidity out of range: 0
Negative wind speed: 0


In [7]:
p90 = air.groupby("City")["PM2.5"].quantile(0.90).rename("PM25_P90_City")
air = air.merge(p90, on="City", how="left")
air["PM25_Spike"] = air["PM2.5"] > air["PM25_P90_City"]

air[["City", "PM2.5", "PM25_P90_City", "PM25_Spike"]].head()


,City,PM2.5,PM25_P90_City,PM25_Spike
0,Bangkok,86.57,138.142,False
1,Istanbul,50.63,136.585,False
2,Rio de Janeiro,130.21,134.430,False
3,Mumbai,119.70,138.613,False
4,Paris,55.20,130.136,False


In [8]:
cty.columns = [c.strip() for c in cty.columns]
cty["Country Name"] = cty["Country Name"].astype(str).str.strip()
cty["Country Code"] = cty["Country Code"].astype(str).str.strip()
cty["Continent Name"] = cty["Continent Name"].astype(str).str.strip()
cty["Year"] = pd.to_numeric(cty["Year"], errors="coerce").astype("Int64")

# keep best row per (Country, Year): fewest nulls
cty["_nulls"] = cty.isna().sum(axis=1)
cty = (cty.sort_values(["Country Name", "Year", "_nulls"])
          .drop_duplicates(subset=["Country Name", "Year"], keep="first")
          .drop(columns=["_nulls"]))

assert cty.duplicated(subset=["Country Name", "Year"]).sum() == 0

# missing flags (useful in report + Tableau)
for col in ["R&D", "Unemployment", "Inflation Rate", "Ease of Doing Business"]:
    if col in cty.columns:
        cty[f"Missing_{col.replace(' ', '_')}"] = cty[col].isna()

cty.isna().sum().sort_values(ascending=False).head(10)


R&D                           1518
Unemployment                   667
Ease of Doing Business         598
Health Expenditure             575
Health Expenditure (% GDP)     575
Inflation Rate                 552
Import                         460
Import (% GDP)                 460
Net Trade                      460
Export                         460
dtype: int64

In [14]:
country_pollution = merged.groupby(["Country_join", "Continent Name"], as_index=False)[pollutants].mean()

country_pollution = country_pollution.merge(
    cty_2022[["Country_join",
              "GDP Per Capita (imputed)", "Industry (% GDP) (imputed)", "Population Density (imputed)",
              "Unemployment (imputed)", "Inflation Rate (imputed)", "GDP (imputed)", "Population (imputed)"]],
    on="Country_join", how="left"
)

def zscore(s):
    mu = s.mean()
    sd = s.std(ddof=0)
    return (s - mu) / sd if sd != 0 else 0

country_pollution["PollutionIndex"] = (
    zscore(country_pollution["PM2.5"]) +
    zscore(country_pollution["PM10"]) +
    zscore(country_pollution["NO2"]) +
    zscore(country_pollution["O3"])
)

country_pollution["EconomicStress"] = (
    zscore(country_pollution["Unemployment (imputed)"]) +
    zscore(country_pollution["Inflation Rate (imputed)"])
)

country_pollution["Quadrant"] = np.select(
    [
        (country_pollution["PollutionIndex"] >= 0) & (country_pollution["EconomicStress"] >= 0),
        (country_pollution["PollutionIndex"] >= 0) & (country_pollution["EconomicStress"] < 0),
        (country_pollution["PollutionIndex"] < 0) & (country_pollution["EconomicStress"] >= 0),
        (country_pollution["PollutionIndex"] < 0) & (country_pollution["EconomicStress"] < 0),
    ],
    [
        "High Pollution / High Stress",
        "High Pollution / Low Stress",
        "Low Pollution / High Stress",
        "Low Pollution / Low Stress",
    ],
    default="Unknown"
)

country_pollution.sort_values("PollutionIndex", ascending=False)


,Country_join,Continent Name,PM2.5,PM10,NO2,SO2,CO,O3,GDP Per Capita (imputed),Industry (% GDP) (imputed),Population Density (imputed),Unemployment (imputed),Inflation Rate (imputed),GDP (imputed),Population (imputed),PollutionIndex,EconomicStress,Quadrant
7,India,Asia,78.901167,105.841667,53.344074,25.029444,5.077556,106.643648,2388.621198,25.620807,431.110765,7.330000,6.699034,3.385090e+12,1.417173e+09,4.158609,-0.143766,High Pollution / Low Stress
8,Japan,Asia,78.867475,105.040937,52.672770,27.978635,5.101446,108.479491,33815.317273,28.911817,331.074836,2.641000,2.497703,4.231141e+12,1.251250e+08,3.344365,-1.181599,High Pollution / Low Stress
11,Russian Federation,Europe,77.878610,105.936286,52.029876,26.110913,5.120996,107.294212,15606.638235,32.793532,8.395934,6.548636,9.925566,2.240422e+12,1.435557e+08,2.310855,-0.051710,High Pollution / Low Stress
12,South Africa,Africa,77.666008,105.135125,53.146929,24.674261,5.093397,106.182898,6776.480077,24.443093,49.129995,29.806000,7.039727,4.058697e+11,5.989388e+07,2.085395,3.494831,High Pollution / High Stress
16,United Arab Emirates,Asia,80.013692,103.891192,52.683038,24.964096,5.084346,106.395673,53757.863251,50.287642,95.705322,2.754000,4.827889,5.075349e+11,9.441129e+06,1.928819,-1.006163,High Pollution / Low Stress
2,Canada,North America,77.827645,103.867799,51.546834,24.802027,4.946544,110.850367,54966.488836,26.864927,3.940373,5.209000,6.802801,2.139840e+12,3.892990e+07,1.144306,-0.477958,High Pollution / Low Stress
1,Brazil,South America,75.669532,105.440876,52.103971,24.578330,4.906721,108.069308,8917.673898,20.697623,25.284090,9.461000,9.280106,1.920096e+12,2.153135e+08,0.688945,0.373227,High Pollution / High Stress
5,France,Europe,74.694622,104.642761,53.818487,24.572127,4.940757,106.910020,40963.837337,17.382818,123.721547,7.445000,5.222367,2.782905e+12,6.793566e+07,0.615682,-0.224923,High Pollution / Low Stress
0,Australia,Oceania,78.933560,103.373951,52.474938,25.684033,5.023477,106.818436,64491.429886,27.875998,3.355923,3.661000,6.594097,1.675419e+12,2.597894e+07,0.571643,-0.741062,High Pollution / Low Stress
18,United States,North America,77.113680,104.288035,52.303326,25.674802,5.121549,105.685572,76398.591742,19.743835,34.164005,3.611000,8.002800,2.546270e+13,3.332876e+08,-0.441549,-0.654035,Low Pollution / Low Stress


In [19]:
air_long = merged.melt(
    id_vars=[c for c in merged.columns if c not in pollutants],
    value_vars=pollutants,
    var_name="Pollutant",
    value_name="Value"
)

air.to_csv(f"{OUT_DIR}/01_air_clean.csv", index=False)
cty.to_csv(f"{OUT_DIR}/02_countries_clean_dedup.csv", index=False)
cty_2022.to_csv(f"{OUT_DIR}/03_countries_2022_dim.csv", index=False)
merged.to_csv(f"{OUT_DIR}/04_air_country_2022_merged.csv", index=False)
air_long.to_csv(f"{OUT_DIR}/05_air_long_pollutants.csv", index=False)
city_avg.to_csv(f"{OUT_DIR}/06_city_avg_snapshot.csv", index=False)
city_month.to_csv(f"{OUT_DIR}/07_city_month_trends.csv", index=False)
country_pollution.to_csv(f"{OUT_DIR}/08_country_pollution_summary_2022.csv", index=False)

air_long.shape


(60000, 52)